In [ ]:
from flytekit.configuration import set_flyte_config_file
set_flyte_config_file("notebook.config")

def print_console_url(exc):
    print("http://localhost:30081/console/projects/{}/domains/{}/executions/{}".format(exc.id.project, exc.id.domain, exc.id.name))

In [ ]:
import boto3
from botocore.client import Config

s3 = boto3.resource('s3',
                    endpoint_url='http://localhost:30084',
                    aws_access_key_id='minio',
                    aws_secret_access_key='miniostorage',
                    config=Config(signature_version='s3v4'),
                    region_name='us-east-1')

def upload_file(f, ref):
    mod = ref.lstrip("s3://")
    bucket, path = mod.split("/", 1)
    s3.Bucket(bucket).upload_file('image.py',path)

In [ ]:
from IPython.display import Image, display
def display_images(paths):
    for p in paths:
        display(Image(p))


# Demo: Remote iteration with an OpenCV Script, using an open source OpenCV Image

We are using the filter_edges method that is available in the adjoining module - image.py
> filter_edges

## Step I: Run the filter_edges example within Jupyter Notebook First

In [ ]:
from image import filter_edges

In [ ]:
inp = "image.jpg" # This should be a path to an image that is available locally 
out = "edges.png" # This is the path to where we want to create the image
filter_edges(inp, out)
# Invoke the helper method that displays the image in Jupyter
display_images([inp, out])

## Step II: Create a Flyte Task for this function
In this case we will use an SdkRawContainerTask. A raw container task is essentially a container task, where we tell Flyte, that this container does not have flytekit. So all the inputs and outputs should be auto-mounted and uploaded.
The task can use an open source container `docker.io/jjanzic/docker-python3-opencv`. This container has python and OpenCV already installed.

### Hot load the code
If you notice the edges task accepts 2 inputs
"image": the image to be converted
"script": the script to execute. It is assumed the script to be executed is available in some bucket (s3) that is accessible by your Flyte task.

### Command
The most important part is the command
```python
["python", "/inputs/script", "/inputs/image", "/outputs/edges"],
```
the command is just running python and the passed in script. Note the input name and output name
The input name is the name of the "input variable". If this was a list of images, then this would be a directory.
The output is also the name of the "output variable"

The names are extremely important, as Flyte will only download the image locally to a file that has the name that matches the variable name. Also it will upload a file whose name matches the output variable name. The extension of the file does not matter.

In [ ]:
from flytekit.common.tasks.raw_container import SdkRawContainerTask
from flytekit.sdk.types import Types

edges = SdkRawContainerTask(
    input_data_dir="/inputs",
    output_data_dir="/outputs",
    inputs={"image": Types.Blob, "script": Types.Blob},
    outputs={"edges": Types.Blob},
    image="docker.io/jjanzic/docker-python3-opencv",
    command=["python", "/inputs/script", "/inputs/image", "/outputs/edges"],
)

   

## Step IIIa: Just for this excercise
To make the dynamic loading of the script work, we have to **upload the script to some s3 bucket**. Since we are testing this locally, I have originally created an s3 client that points to Flyte installed minio. Upload the code there too


In [ ]:
script_path="s3://my-s3-bucket/code/image.py"
upload_file("image.py", script_path)
example_image="https://www.naturephotographysimplified.com/wp-content/uploads/2019/06/How-to-get-sharp-images-Birds-in-flight-Bharatpur-Bird-Sanctuary-bird-Photography-by-Prathap-DK-bronze-winged-jacana-Greater-Spotted-Eagle-750x500.jpg"

## Step III: Launch an execution for the task
This creates an execution of just the task. Remember in Flyte, Task is a standalone top level entity, so you can execute it

In [ ]:
exc = edges.register_and_launch("flyteexamples", "development", inputs={"image":example_image, "script":script_path})

print_console_url(exc)


In [ ]:
exc.wait_for_completion()
#exc.outputs
# Outputs are not working with Minio

In [ ]:
from flytekit.sdk.workflow import workflow_class, Input, Output
@workflow_class
class EdgeDetector(object):
    script = Input(Types.Blob)
    image = Input(Types.Blob)
    edge_task = edges(script=script, image=image)
    out =  Output(edge_task.outputs.edges, sdk_type=Types.Blob)

EdgeDetector_lp = EdgeDetector.create_launch_plan() 

## Step IV: Optional. Create a Workflow
Ofcourse you can use this task in a workflow. We are creating a trivial workflow in this case that has only one task

## Step V: Optional. Register and execute the workflow
To make the dynamic loading of the script work, we have to upload the script to some s3 bucket. Since we are testing this locally, I have originally created an s3 client that points to Flyte installed minio. Upload the code there too
```python
s3.Bucket('my-s3-bucket').upload_file('image.py','code/image.py')
```

In [ ]:
edges.register(name="EdgeDetectorFunc", project="flyteexamples", domain="development", version="5")
EdgeDetector.register(name="EdgeDetector", project="flyteexamples", domain="development", version="5")
EdgeDetector_lp.register(name="EdgeDetector", project="flyteexamples", domain="development", version="5")

In [ ]:
exc = EdgeDetector_lp.execute("flyteexamples", "development", inputs={"image":example_image, "script":script_path})
print_console_url(exc)

## Step VI: Visualize the results
You can retrieve the results and visualize them here

In [ ]:
key="/gn/ff554920363ff4da1903-edge-task-0/edges"
s3.Bucket('my-s3-bucket').download_file(key,'edges.png')

In [ ]:
display_images(["edges.png"])